In [ ]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [ ]:
load_dotenv()

In [ ]:
conn_string = (
    f"host={os.getenv('hostname')} "
    f"dbname={os.getenv('database')} "
    f"user={os.getenv('username')} "
    f"password={os.getenv('password')} "
    f"port={os.getenv('port')}"
)

In [ ]:
conn = psycopg2.connect(conn_string)

In [ ]:
cursor = conn.cursor()

In [ ]:
schema_name = "public,dadeh"

In [ ]:
cursor.execute(f"SET search_path TO {schema_name}")

In [ ]:
complement_dispositif_public = []
for i in range(2,8):
    complement_dispositif_public.append("0" + str(i))

In [ ]:
complement_dispositif_public

In [ ]:
counts_dispositif = []
for cdp in complement_dispositif_public:
    print(cdp)
    mydict = {}
    mydict["cdp"] = cdp
    query = f"select count(*) from ddadtcontrat where complement_dispositif_public = '{cdp}';"
    df = pd.read_sql_query(query, conn)
    mydict["count"] = df["count"].iloc[0]
    counts_dispositif.append(mydict)

In [ ]:
counts_dispositif[:1]

In [ ]:
df.shape

In [ ]:
counts_dispositif

In [ ]:
counts_dispositif[1:]

In [ ]:
for cd in counts_dispositif[1:]:
    print(cd["cdp"])
    df = pd.DataFrame([])
    query = (
        "select * from ddadtcontrat where complement_dispositif_public = '" + \
        cd['cdp'] + "';"
    )
    df = pd.read_sql_query(query, conn)
    df.to_csv("cdp_" + cd["cdp"] + ".csv", index=False)

In [ ]:
for cd in counts_dispositif:
    print(cd["cdp"])
    df = pd.DataFrame([])
    query = (
        "select c.id, c.id_employeur_assure, ea.id_assure, ea.id_employeur from ddadtcontrat c " 
        "left join dadeh.ddadtemployeur_assure ea on ea.id = c.id_employeur_assure "
        "where c.complement_dispositif_public = '" + cd["cdp"] + "';"
    )
    df = pd.read_sql_query(query, conn)
    df.to_csv("cdp_join_" + cd["cdp"] + ".csv", index=False)

In [ ]:
df = pd.DataFrame([])
df2 = pd.DataFrame([])
for cd in counts_dispositif:
    dfinter = pd.read_csv("cdp_" + cd["cdp"] + ".csv", dtype=str)
    dfinter2 = pd.read_csv("cdp_join_" + cd["cdp"] + ".csv", dtype=str)
    df = pd.concat([df, dfinter])
    df2 = pd.concat([df2, dfinter2])

In [ ]:
df

In [ ]:
df = pd.merge(df, df2[["id_employeur_assure", "id_assure", "id_employeur"]], how="left", on="id_employeur_assure")

In [ ]:
df.to_csv("cdp_global.csv", index=False)

In [ ]:
df.id_assure.nunique()

In [ ]:
assures_insertion = list(df.id_assure.unique())

In [ ]:
len(assures_insertion)

In [ ]:
dfc = pd.DataFrame([])

In [ ]:
i = 0
cpt = 0
for em in employeurs:
    if cpt == 0:
        str_em = ""
    cpt += 1
    str_em += "id = " + em + " OR "
    if cpt == 100:
        # send
        str_em = str_em[:-3]
        query = (
            "select id, et_code_apet, et_ad_code_postal, et_categorie_juridique, "
            "et_effectif_fin_periode, en_code_apen, en_ad_code_postal, en_effectif_moyen "
            "from ddadtemployeur where " + str_em
        )
        dfinter = pd.read_sql_query(query, conn)
        dfe = pd.concat([dfe, dfinter])
        cpt = 0
        i = i + 1
        print(i * 100)

In [ ]:
i = 0
cpt = 0
for ai in assures_insertion[28115:]:
    if cpt == 0:
        str_ai = ""
    cpt += 1
    str_ai += "ea.id_assure = " + ai + " OR "
    if cpt == 100:
        str_ai = str_ai[:-3]
        query = (
            "select c.id, c.id_employeur_assure, c.date_debut_contrat, c.statut_conventionnel, "
            "c.statut_rc, c.pcs_ese, c.code_complement_pcs_ese, c.libelle_emploi, c.nature_contrat, "
            "c.dispositif_politique, c.num_contrat, c.date_fin_previsionnelle, c.unite_mesure, "
            "c.quotite_categorie, c.quotite, c.modalite_temps, c.complement_base, c.ccn, c.regime_maladie, "
            "c.id_lieu_travail, c.regime_vieillesse, c.motif_recours, c.code_caisse_conges_payes, "
            "c.travailleur_etranger, c.taux_frais_professionnels, c.statut_emploi, c.motif_exclusion_dsn, "
            "c.code_affectation_ac, c.num_interne, c.type_gestion_ac, c.date_adhesion, "
            "c.date_convention_gestion, c.num_convention_gestion, c.code_gestionnaire_risque, c.code_metier, "
            "c.regime_risque_accident_travail, c.code_risque_accident_travail, c.positionnement_cc, "
            "c.code_statut_categoriel_apecita, c.taux_accident_travail, c.salarie_tpartiel_cotis_tplein, "
            "c.remuneration_pourboir, c.id_lieu_etab_utilisateur, c.num_certification_sociale, c.num_label, "
            "c.num_licence_entrep_spectacle, c.num_objet_spectacle, c.statut_organisateur_spectacle, "
            "c.date_fin_contrat, c.motif_rupture, c.dernier_jour_travaille, c.date_dernieres_infos, "
            "c.mod_decl_fin_cont_usage, c.activite_partielle, c.statut_boeth, c.mise_a_dispo_ext, "
            "c.cat_classement_finale, c.college_cnieg, c.complement_dispositif_public, c.id_contrat_em, "
            "c.mois_declaration, c.grade, c.finess_geographique, c.echelon, c.coef_hierarchique, "
            "c.flag_traitement_vue_donnee, ea.id_assure, ea.id_employeur "
            "from ddadtcontrat c "
            "left join dadeh.ddadtemployeur_assure ea "
            "on ea.id = c.id_employeur_assure "
            "where " + str_ai + ";"
        )
        dfinter = pd.read_sql_query(query, conn)
        dfc = pd.concat([dfc, dfinter])        
        cpt = 0
        i = i + 1
        print(28115 + i * 100)

In [ ]:
str_ai = str_ai[:-3]
query = (
    "select c.id, c.id_employeur_assure, c.date_debut_contrat, c.statut_conventionnel, "
    "c.statut_rc, c.pcs_ese, c.code_complement_pcs_ese, c.libelle_emploi, c.nature_contrat, "
    "c.dispositif_politique, c.num_contrat, c.date_fin_previsionnelle, c.unite_mesure, "
    "c.quotite_categorie, c.quotite, c.modalite_temps, c.complement_base, c.ccn, c.regime_maladie, "
    "c.id_lieu_travail, c.regime_vieillesse, c.motif_recours, c.code_caisse_conges_payes, "
    "c.travailleur_etranger, c.taux_frais_professionnels, c.statut_emploi, c.motif_exclusion_dsn, "
    "c.code_affectation_ac, c.num_interne, c.type_gestion_ac, c.date_adhesion, "
    "c.date_convention_gestion, c.num_convention_gestion, c.code_gestionnaire_risque, c.code_metier, "
    "c.regime_risque_accident_travail, c.code_risque_accident_travail, c.positionnement_cc, "
    "c.code_statut_categoriel_apecita, c.taux_accident_travail, c.salarie_tpartiel_cotis_tplein, "
    "c.remuneration_pourboir, c.id_lieu_etab_utilisateur, c.num_certification_sociale, c.num_label, "
    "c.num_licence_entrep_spectacle, c.num_objet_spectacle, c.statut_organisateur_spectacle, "
    "c.date_fin_contrat, c.motif_rupture, c.dernier_jour_travaille, c.date_dernieres_infos, "
    "c.mod_decl_fin_cont_usage, c.activite_partielle, c.statut_boeth, c.mise_a_dispo_ext, "
    "c.cat_classement_finale, c.college_cnieg, c.complement_dispositif_public, c.id_contrat_em, "
    "c.mois_declaration, c.grade, c.finess_geographique, c.echelon, c.coef_hierarchique, "
    "c.flag_traitement_vue_donnee, ea.id_assure, ea.id_employeur "
    "from ddadtcontrat c "
    "left join dadeh.ddadtemployeur_assure ea "
    "on ea.id = c.id_employeur_assure "
    "where " + str_ai + ";"
)
dfinter = pd.read_sql_query(query, conn)
dfc = pd.concat([dfc, dfinter])

In [ ]:
dfc.shape

In [ ]:
dfc.to_csv("contrat_inseres.csv", index=False)